In [11]:
from pathlib import Path

BASE = Path.cwd()
PDF_DIR = BASE / "data" / "pdfs"
PROCESSED_DIR = BASE / "data" / "processed"
INDEX_DIR = BASE / "index" / "chroma_autos"

for p in [PDF_DIR, PROCESSED_DIR, INDEX_DIR]:
    p.mkdir(parents=True, exist_ok=True)

PDF_DIR  # Coloca aquí tus fichas PDF (por marca/modelo)


PosixPath('/home/pibezx/Documents/Proyectos/data/pdfs')